# 自製智能中文選字系統  (1)

In [13]:
import sys
sys.version

'3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 22:22:05) [MSC v.1916 64 bit (AMD64)]'

## 資料前處理

確認版本為 python3

In [14]:
import re

In [16]:
# \u4E00-\u9FFF 代表常見中文字，搭配Regex去只選取中文
re.findall(pattern='[\u4E00-\u9FFF]', string='“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')

['英',
 '語',
 '一',
 '詞',
 '源',
 '於',
 '遷',
 '居',
 '英',
 '格',
 '蘭',
 '的',
 '日',
 '耳',
 '曼',
 '部',
 '落',
 '盎',
 '格',
 '魯',
 '而',
 '盎',
 '格',
 '魯',
 '得',
 '名',
 '於']

In [15]:
# \u4E00-\u9FFF 代表常見中文字，搭配Regex去只選取中文
re.findall(pattern='[\u4E00-\u9FFF]+', string='“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [17]:
# 透過finditer可以得到match物件以及其span, 就可以去判斷在哪
string = '“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於'
for e in re.finditer(pattern='[\u4E00-\u9FFF]', string=string):
    print(e)
    print(e.span())
    print(string[e.span()[0]])

<re.Match object; span=(1, 2), match='英'>
(1, 2)
英
<re.Match object; span=(2, 3), match='語'>
(2, 3)
語
<re.Match object; span=(4, 5), match='一'>
(4, 5)
一
<re.Match object; span=(5, 6), match='詞'>
(5, 6)
詞
<re.Match object; span=(6, 7), match='源'>
(6, 7)
源
<re.Match object; span=(7, 8), match='於'>
(7, 8)
於
<re.Match object; span=(8, 9), match='遷'>
(8, 9)
遷
<re.Match object; span=(9, 10), match='居'>
(9, 10)
居
<re.Match object; span=(10, 11), match='英'>
(10, 11)
英
<re.Match object; span=(11, 12), match='格'>
(11, 12)
格
<re.Match object; span=(12, 13), match='蘭'>
(12, 13)
蘭
<re.Match object; span=(13, 14), match='的'>
(13, 14)
的
<re.Match object; span=(14, 15), match='日'>
(14, 15)
日
<re.Match object; span=(15, 16), match='耳'>
(15, 16)
耳
<re.Match object; span=(16, 17), match='曼'>
(16, 17)
曼
<re.Match object; span=(17, 18), match='部'>
(17, 18)
部
<re.Match object; span=(18, 19), match='落'>
(18, 19)
落
<re.Match object; span=(19, 20), match='盎'>
(19, 20)
盎
<re.Match object; span=(20, 21), match='

In [18]:
# nice!
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    matches = list(re.finditer(pattern='[\u4E00-\u9FFF]', string=line))
    segments = []
    if len(matches) >= 1:
        # 先把開頭補上去
        chars = [line[matches[0].span()[0]]]
    else:
        chars = []
    matches = list(matches)
    for i in range(len(matches)-1):
        # 如果前後的span是連在一起的就接起來, 否則就是隔開
        if matches[i+1].span()[0] - matches[i].span()[1] == 0:
            chars.append(line[matches[i].span()[1]])
        else:
            segments.append(''.join(chars))
            chars = [line[matches[i+1].span()[0]]]
    # 最後一個!
    if chars:
        segments.append(''.join(chars))
    
    # END YOUR CODE
    return segments

In [19]:
def prepocess_line(line):
    # 僅僅挑出中文字元，並且斷開不連續的中文字
    # YOUR CODE HERE
    chinese_pattern = r'[\u4E00-\u9FFF]+'
    segments = re.findall(chinese_pattern, line, flags=re.UNICODE)
    # END YOUR CODE
    return segments

In [20]:
prepocess_line('“英語”一詞源於遷居英格蘭的日耳曼部落盎格魯（），而“盎格魯”得名於')  
# 應該為：['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

['英語', '一詞源於遷居英格蘭的日耳曼部落盎格魯', '而', '盎格魯', '得名於']

In [21]:
segments = []
with open('./datasets/wiki_zh_small.txt', encoding='utf8') as fr:
    for line in fr.readlines():
        segments += prepocess_line(line)

In [22]:
segments[:10]

['英語',
 '英語英語',
 '又稱爲英文',
 '是一種西日耳曼語言',
 '誕生於中世紀早期的英格蘭',
 '如今具有全球通用語的地位',
 '英語',
 '一詞源於遷居英格蘭的日耳曼部落盎格魯',
 '而',
 '盎格魯']

In [23]:
from collections import Counter

Counter(segments)

Counter({'英語': 8,
         '英語英語': 1,
         '又稱爲英文': 1,
         '是一種西日耳曼語言': 1,
         '誕生於中世紀早期的英格蘭': 1,
         '如今具有全球通用語的地位': 1,
         '一詞源於遷居英格蘭的日耳曼部落盎格魯': 1,
         '而': 37,
         '盎格魯': 3,
         '得名於臨波羅的海的半島盎格里亞': 1,
         '弗裏西語是與英語最相近的語言': 1,
         '英語詞彙在中世紀早期受到了其他日耳曼族語言的大量影響': 1,
         '後來受羅曼族語言尤其是法語的影響': 1,
         '英語是將近六十個國家唯一的官方語言或官方語言之一': 1,
         '也是全世界最多國家的官方語言': 1,
         '它是英國': 1,
         '愛爾蘭': 2,
         '美國': 16,
         '加拿大': 9,
         '澳大利亞和新西蘭最常用的語言': 1,
         '也在加勒比': 1,
         '非洲及南亞的部分地區被廣泛使用': 1,
         '英語是世界上母語人口第三多的語言': 1,
         '僅次於漢語和西班牙語': 1,
         '英語是學習者最多的第一外語': 1,
         '是聯合國': 1,
         '歐盟和許多其他國際組織的官方語言': 1,
         '它是使用最廣泛的日耳曼族語言': 1,
         '至少': 4,
         '的日耳曼語族使用者說英語': 1,
         '英語有': 1,
         '多年的發展史': 1,
         '公元': 17,
         '世紀': 54,
         '撒克遜人把他們的各種盎格魯': 1,
         '弗裏西語方言帶到了大不列顛島': 1,
         '它們被稱爲古英語': 1,
         '中古英語始於': 1,
         '世紀後期的諾曼征服': 1,
 

## Ngram

一開始要先計算字詞出現的次數

In [24]:
# 熟悉Counter操作
from collections import Counter

c = Counter()
c['你好'] += 1    # 可以直接這樣做!
print(c)
print(Counter('你好') + Counter('hi'))

Counter({'你好': 1})
Counter({'你': 1, '好': 1, 'h': 1, 'i': 1})


In [29]:
from collections import Counter

class Counters:
    def __init__(self, n):
        self.n = n
        self.counters = [Counter() for _ in range(n + 1)]  # 分別代表計算0、1、...個字的出現次數(這個沒解釋真的是看不懂, 代表看的n-1個字)

    def fit(self, segments: list):
        # 因為 self.counters 分別代表計算0、1、...個字的出現次數
        # 請在此實作利用 segments 以及函式 _skip 來統計次數(這邊是原敘述，寫得有點含糊，是說，要將每一句話都去計算Counter, 不用斷詞)
        # YOUR CODE HERE
        for i in range(1, self.n+1):    # 此時i代表看的字數
            for segment in segments:
                # 每一句都去算!Counter可以直接相加
                self.counters[i] += Counter(self._skip(segment, i))
        base_count = sum(dict(self.counters[1]).values())    # 總共有多少字, 為什麼要有這個!?
        self.counters[0] = Counter({'': base_count})
        # END YOUR CODE

    def __getitem__(self, k):
        """
            出現該
        """
        return self.counters[k]

    def _skip(self, segment, n):
        """
            這邊是return所有句子(segment)的看n周圍的word有哪些
        """
        assert n > 0
        if len(segment) < n:
            return []
        shift = n - 1
        for i in range(len(segment) - shift):
            yield segment[i:i+shift+1]

In [30]:
counters = Counters(n=3)
counters.fit(segments)

In [31]:
counters[0]
# 應該為： Counter({'': 371373})

Counter({'': 371373})

In [38]:
class Ngram:
    def __init__(self, n, counters):
        assert n <= counters.n
        self.n = n
        self.major_counter = counters[n]    # 只有n跟n-1是因為Ngram這個演算法是根據前n-1個字, 去看第n個字，故只需要(n-1)這個長度的聯合出現次數與n長度的聯合出現次數
        self.minor_counter = counters[n-1]

    def predict_proba(self, prefix='', top_k=5):
        """
            這是根據prefix去預測下一個字出現機率, 只是吃單一個輸入
        """
        assert len(prefix) >= self.n - 1
        # 使用 Ngram 的公式計算出下一個字出現的機率
        # 輸出為機率與字的tuple列表，詳見下方輸出範例
        # YOUR CODE HERE
        reference = prefix[-(self.n-1):] if self.n > 1 else ''    # 這個是真正的條件機率的條件部分
        count_reference = self.minor_counter[reference]         # 這是該prefix(真正的)的出現次數
        probs = []
        for key, count in self.major_counter.items():  # key:count==字:字的出現次數
            if key.startswith(reference):
                prob = count / count_reference
                probs.append((prob, key[-1]))    # -1代表最後, 也就是最新看到的字, 如: "我愛你", reference="我愛", key[-1]==我
        
        sorted_probs = sorted(probs, key=lambda x: x[0], reverse=True)
        # END YOUR CODE
        return sorted_probs[:top_k] if top_k > 0 else sorted_probs

    def get_proba_dict(self, prefix=''):
        return {word: prob for prob, word in self.predict_proba(prefix, top_k=-1)}


In [39]:
unigram = Ngram(1, counters)

In [40]:
unigram.predict_proba('我思')
# 應該為：[(0.035732269174118744, '的'),
#         (0.012927703414087723, '國'),
#         (0.010620050461395955, '中'),
#         (0.009984570768472667, '在'),
#         (0.009852627950874188, '一')]

[(0.035732269174118744, '的'),
 (0.012927703414087723, '國'),
 (0.010620050461395955, '中'),
 (0.009984570768472667, '在'),
 (0.009852627950874188, '一')]

In [44]:
bigram = Ngram(2, counters)
trigram = Ngram(3, counters)

## 使用Ngram來建立第一版選字系統

In [45]:
class ChineseWordRecommenderV1:
    def __init__(self, unigram, bigram, trigram):
        self.unigram = unigram
        self.bigram = bigram
        self.trigram = trigram
    
    def predict_proba(self, prefix='', top_k=5):
        # 使用Ngram來建立選字系統
        # YOUR CODE HERE
        length = len(prefix)
        if length > 1:
            return self.trigram.predict_proba(prefix, top_k)
        elif length > 0:
            return self.bigram.predict_proba(prefix, top_k)
        else:
            return self.unigram.predict_proba(prefix, top_k)
        
        # END YOUR CODE

In [46]:
model = ChineseWordRecommenderV1(unigram, bigram, trigram)

In [47]:
probs = model.predict_proba('我思', top_k=10)
probs

[(0.75, '故'), (0.25, '維')]

## Demo

In [48]:
!pip install -U pip
!pip install -q ipywidgets

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: pip
    Found existing installation: pip 21.0
    Uninstalling pip-21.0:
      Successfully uninstalled pip-21.0


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [49]:
import ipywidgets as widgets

text = widgets.Textarea()
label = widgets.Label()
display(label, text)

def func(change):
    probs = model.predict_proba(change.new, top_k=10)
    label.value = ' ' + '\t'.join([word for prob, word in probs])

text.observe(func, names='value')

Label(value='')

Textarea(value='')